In [1]:
import copy
import itertools as its
import os
import pathlib
from typing import Dict, List, Optional
from collections import defaultdict

import networkx as nx
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

from aoc import sim, testing

%matplotlib inline

INPUT_PATH = pathlib.Path('..') / 'input' / 'dec07.txt'

In [2]:
test_input1 = '3,15,3,16,1002,16,10,16,1,16,15,15,4,15,99,0,0'
test_input2 = '3,23,3,24,1002,24,10,24,1002,23,-1,23,101,5,23,23,1,24,23,23,4,23,99,0,0'
test_input3 = '3,31,3,32,1002,32,10,32,1001,31,-2,31,1007,31,0,33,1002,33,7,33,1,33,31,31,1,32,31,31,4,31,99,0,0,0'

In [3]:
test_ops1 = sim.read_ops(test_input1)
test_ops2 = sim.read_ops(test_input2)
test_ops3 = sim.read_ops(test_input3)

In [4]:
class Capture:
    def __init__(self, output: Optional[int] = None, retval: bool = True):
        self.output = output
        self.retval = retval

    def __call__(self, output: int) -> bool:
        self.output = output
        return self.retval

def get_max_output(ops: List[int]) -> int:
    max_output = -float('inf')
    for settings in its.permutations([0, 1, 2, 3, 4]):
        cap = Capture(0)
        for setting in settings:
            sim.simulate(ops, inputs=[setting, cap.output], output_func=cap)
        max_output = max(max_output, cap.output)
    return max_output

In [5]:
assert get_max_output(test_ops1) == 43210
assert get_max_output(test_ops2) == 54321
assert get_max_output(test_ops3) == 65210

In [6]:
ops = sim.read_ops(INPUT_PATH.read_text())
print(f'The answer for part 1 is {get_max_output(ops)}')

The answer for part 1 is 17440


In [7]:
def get_max_feedback_output(ops: List[int]) -> int:
    max_output = -float('inf')
    for settings in its.permutations([5, 6, 7, 8, 9]):
        cap = Capture(0)
        all_ops = [copy.copy(ops) for _ in range(5)]
        pids = [0, 0, 0, 0, 0]
        
        for i, (setting, my_ops) in enumerate(zip(settings, all_ops)):
            pid = pids[i]
            pid = sim.simulate(my_ops, inplace=True, inputs=[setting, cap.output], output_func=cap, starting_pid=pid)
            pids[i] = pid
            if pid == -1:
                break

        for i, my_ops in enumerate(its.cycle(all_ops)):
            pid = pids[i % 5]
            pid = sim.simulate(my_ops, inplace=True, inputs=[cap.output], output_func=cap, starting_pid=pid)
            pids[i % 5] = pid
            if pid == -1:
                break

        max_output = max(max_output, cap.output)
    return max_output

In [8]:
test_input4 = '3,26,1001,26,-4,26,3,27,1002,27,2,27,1,27,26,27,4,27,1001,28,-1,28,1005,28,6,99,0,0,5'
test_input5 = '3,52,1001,52,-5,52,3,53,1,52,56,54,1007,54,5,55,1005,55,26,1001,54,-5,54,1105,1,12,1,53,54,53,1008,54,0,55,1001,55,1,55,2,53,55,53,4,53,1001,56,-1,56,1005,56,6,99,0,0,0,0,10'

In [9]:
test_ops4 = sim.read_ops(test_input4)
test_ops5 = sim.read_ops(test_input5)

assert get_max_feedback_output(test_ops4) == 139629729
assert get_max_feedback_output(test_ops5) == 18216

In [10]:
print(f'The answer to part 2 is {get_max_feedback_output(ops)}')

The answer to part 2 is 27561242
